# **Creating CNN (Convolution Neural Network) Model Using Keras**

#### *Use Google colab (with runtime as GPU) to run this because in normal cpu it will take more time*

In [27]:
!pip install keras-tuner

In [28]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [29]:
print(tf.__version__)

2.5.0


In [30]:
fashion_mnist = keras.datasets.fashion_mnist

In [31]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [32]:
train_images.shape

(60000, 28, 28)

In [33]:
train_labels.shape

(60000,)

In [34]:
test_images.shape

(10000, 28, 28)

In [35]:
test_labels.shape

(10000,)

In [36]:
train_images = train_images/255.0
test_images = test_images/255.0

In [37]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [38]:
train_images.shape

(60000, 28, 28, 1)

In [39]:
train_images[0].shape

(28, 28, 1)

In [40]:
def build_model(hp):
    model = keras.Sequential([
        
        # creating th first filter
        keras.layers.Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
            activation='relu',
            input_shape=(28,28,1)
        ),
            
        # creating th second filter
        keras.layers.Conv2D(
            filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
            kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
            activation='relu'
        ),
        
        # Flattening the layer
        keras.layers.Flatten(),
            
        # creating the first hidden layer in that no of neurons will be from 32 to 128
        keras.layers.Dense(
             units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step = 16),
             activation = 'relu'),
            
        # creating the last output layer, we used 10 neurons because in target or label we have 10 diffrent categories
        keras.layers.Dense(10, activation = 'softmax')
    ])
    
# taken adam optimizer, and learning rate is varying from 1e2 and 1e3, and loss is sparse_categorical_crossentropy and metrix is accuracy for multi class classification
    model.compile(
        optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3])),
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']) # taken adam optimizer, and learning rate is varying from 1e2 and 1e3
    
    return model   

In [41]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [42]:
tuner_search = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='project',
    project_name='Mnist Fashion')

In [43]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.8644999861717224

Best val_accuracy So Far: 0.9141666889190674
Total elapsed time: 00h 03m 08s
INFO:tensorflow:Oracle triggered exit


In [44]:
model = tuner_search.get_best_models(num_models=1)[0]

In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        76832     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               1982592   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 2,061,674
Trainable params: 2,061,674
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1340 - accuracy: 0.9501 - val_loss: 0.2682 - val_accuracy: 0.9095
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0950 - accuracy: 0.9640 - val_loss: 0.3128 - val_accuracy: 0.9097
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0648 - accuracy: 0.9763 - val_loss: 0.3307 - val_accuracy: 0.9152
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0461 - accuracy: 0.9829 - val_loss: 0.4199 - val_accuracy: 0.9123
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0325 - accuracy: 0.9883 - val_loss: 0.5027 - val_accuracy: 0.9052
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0279 - accuracy: 0.9906 - val_loss: 0.5146 - val_accuracy: 0.9057
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0257 - accuracy: 0.9912 - val_loss: 0.5794 - val_accuracy